In [38]:
import json
import urllib
import zipfile
import io
import numpy as np

# Load dataset
SW_DATASET_URL = 'https://github.com/SeijiEmery/shape-net-data/raw/master/car-vertex-data/shrinkwrap-data.json.zip'
with urllib.request.urlopen(SW_DATASET_URL) as f:
    with zipfile.ZipFile(io.BytesIO(f.read()), 'r') as zf:
        with zf.open('shrinkwrap-data.json', 'r') as f:
            sw_dataset = json.loads(f.read())
            
sw_keys = list(sw_dataset.keys())
sw_dimensions = set(map(len, sw_dataset.values()))
print("dataset members have the following lengths: %s (expected 294)"%(sw_dimensions,))

sw_keys = [ key for key in sw_dataset.keys() if len(sw_dataset[key]) == 294 ]
non_corrupt_values = [ x for x in sw_dataset.values() if len(x) == 294 ]
corrupt_values = [ x for x in sw_dataset.values() if len(x) != 294 ]
print("Has %s corrupt value(s), %s values, %s keys"%(
    len(corrupt_values), len(non_corrupt_values), len(sw_keys)))

sw_data = np.array(non_corrupt_values)
print("dataset size: %s"%(sw_data.shape,))

x_train, x_test = np.split(sw_data, [500], 0)
print("partitioning as train = %s, test = %s"%(x_train.shape, x_test.shape))

dataset members have the following lengths: {1158, 294} (expected 294)
Has 1 corrupt value(s), 601 values, 601 keys
dataset size: (601, 294)
partitioning as train = (500, 294), test = (101, 294)


In [44]:
from keras.models import Sequential
from keras.layers import Input, Dense, Activation
from keras.models import Model, load_model
import os

INPUT_DIM = 294
HIDDEN_DIM = 80
LATENT_DIM = 10

model = Sequential([
    Dense(HIDDEN_DIM, input_shape=(INPUT_DIM,)),
    Activation('relu'),
    Dense(LATENT_DIM),
    Activation('relu'),
    Dense(HIDDEN_DIM),
    Activation('relu'),
    Dense(INPUT_DIM),
    Activation('linear')
])
trained_epochs = 0
model.compile(optimizer='adam', loss='mse')
model.summary()

def train (epochs, batch_size=50):
    global model, trained_epochs
    print("Training: epoch %s -> %s"%(trained_epochs, epochs + trained_epochs))
    model.fit(x_train, x_train,
             epochs=epochs,
             batch_size=batch_size)
    trained_epochs += epochs
    if not os.path.exists('altmodel'):
        os.makedirs('altmodel')
    path = os.path.join('altmodel', 'altmodel-epoch-%s.h5'%(trained_epochs,))
    print("Saving snapshot as %s"%path)
    model.save(path)
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 80)                23600     
_________________________________________________________________
activation_21 (Activation)   (None, 80)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 10)                810       
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 80)                880       
_________________________________________________________________
activation_23 (Activation)   (None, 80)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 294)               23814     
__________

In [47]:
for i in range(30):
    train(10)

Training: epoch 210 -> 220
Epoch 1/10
500/500 [==============================] - 0s 48us/step - loss: 9.5483e-04
Epoch 2/10
500/500 [==============================] - 0s 57us/step - loss: 9.5620e-04
Epoch 3/10
500/500 [==============================] - 0s 64us/step - loss: 9.5454e-04
Epoch 4/10
500/500 [==============================] - 0s 50us/step - loss: 9.5621e-04
Epoch 5/10
500/500 [==============================] - 0s 60us/step - loss: 9.5354e-04
Epoch 6/10
500/500 [==============================] - 0s 68us/step - loss: 9.5418e-04
Epoch 7/10
500/500 [==============================] - 0s 70us/step - loss: 9.5680e-04
Epoch 8/10
500/500 [==============================] - 0s 73us/step - loss: 9.5606e-04
Epoch 9/10
500/500 [==============================] - 0s 67us/step - loss: 9.5411e-04
Epoch 10/10
500/500 [==============================] - 0s 72us/step - loss: 9.5281e-04
Saving snapshot as altmodel/altmodel-epoch-220.h5
Training: epoch 220 -> 230
Epoch 1/10
500/500 [===============

500/500 [==============================] - 0s 52us/step - loss: 9.3347e-04
Epoch 9/10
500/500 [==============================] - 0s 56us/step - loss: 9.2886e-04
Epoch 10/10
500/500 [==============================] - 0s 49us/step - loss: 9.2795e-04
Saving snapshot as altmodel/altmodel-epoch-300.h5


In [65]:
def predict (epoch):
    path = os.path.join('altmodel', 'altmodel-epoch-%s.h5'%(epoch,))
    print("Loading %s"%path)
    model = load_model(path)
    print("Generating prediction")
    return map(model.predict, (x_train, x_test))
y_train, y_test = predict(200)
y_train.shape, y_test.shape

Loading altmodel/altmodel-epoch-200.h5
Generating prediction


((500, 294), (101, 294))

In [48]:
y_train, y_test = model.predict(x_train), model.predict(x_test)

In [49]:
x_train[0]

array([-0.188041,  0.071757, -0.041727, -0.191469,  0.008376,  0.29926 ,
       -0.186761,  0.076401, -0.039019, -0.191926,  0.010706, -0.317276,
       -0.176773, -0.078195, -0.001716, -0.1874  , -0.107019,  0.266355,
       -0.19355 ,  0.027168,  0.255872, -0.193178,  0.020274, -0.28742 ,
       -0.1874  , -0.118797, -0.285778,  0.002345, -0.018081, -0.446181,
       -0.139854, -0.006102, -0.403974,  0.001501,  0.021966, -0.43595 ,
        0.144545, -0.006102, -0.403974,  0.002345, -0.066678, -0.445537,
       -0.129798, -0.059359, -0.40573 , -0.140946,  0.028435, -0.388389,
        0.145285,  0.028404, -0.388583,  0.132084, -0.059637, -0.406831,
        0.192732,  0.071757, -0.041727,  0.196616,  0.010706, -0.317276,
        0.191451,  0.076401, -0.039019,  0.19616 ,  0.008376,  0.29926 ,
        0.181463, -0.078195, -0.001712,  0.19209 , -0.118797, -0.286042,
        0.197869,  0.020274, -0.28742 ,  0.198237,  0.027142,  0.256007,
        0.19209 , -0.107019,  0.26609 ,  0.001707, 

In [50]:
y_train[0]

array([-1.90569937e-01,  2.16046330e-02, -2.42814086e-02, -1.83027416e-01,
       -5.75248152e-04,  3.04515243e-01, -1.83217198e-01,  6.07237145e-02,
       -3.82540114e-02, -1.83499277e-01, -1.13823311e-02, -3.08607608e-01,
       -1.74083784e-01, -6.93430454e-02, -3.85557255e-03, -1.81659818e-01,
       -9.26676616e-02,  2.62888491e-01, -1.77214965e-01,  5.37756905e-02,
        2.89658010e-01, -1.78229228e-01,  3.21672484e-02, -2.69728482e-01,
       -1.81090280e-01, -9.46186632e-02, -2.68231452e-01,  3.77157331e-03,
       -2.22427957e-02, -4.29328889e-01, -1.35522544e-01, -2.63939966e-02,
       -4.06195939e-01, -5.12617640e-04,  4.54471121e-03, -4.19039786e-01,
        1.32203668e-01, -2.92866174e-02, -4.10046130e-01,  1.58825703e-03,
       -6.69832081e-02, -4.21616077e-01, -1.27337530e-01, -6.67602643e-02,
       -4.04059321e-01, -1.36281550e-01, -8.13290011e-04, -3.94701898e-01,
        1.35797545e-01,  4.93134093e-03, -3.95223409e-01,  1.30829513e-01,
       -6.47926629e-02, -

In [59]:
delta = x_train[0] - y_train[0]
delta / x_train[0]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


array([-1.34488608e-02,  6.98919507e-01,  4.18088800e-01,  4.40885132e-02,
        1.06867815e+00, -1.75607919e-02,  1.89750662e-02,  2.05197386e-01,
        1.96055403e-02,  4.39061063e-02,  2.06317309e+00,  2.73212973e-02,
        1.52128197e-02,  1.13203589e-01, -1.24683715e+00,  3.06306420e-02,
        1.34100846e-01,  1.30146190e-02,  8.43969764e-02, -9.79376122e-01,
       -1.32042623e-01,  7.73834098e-02, -5.86625650e-01,  6.15528433e-02,
        3.36697950e-02,  2.03526493e-01,  6.13992277e-02, -6.08346825e-01,
       -2.30175083e-01,  3.77696750e-02,  3.09712673e-02, -3.32546649e+00,
       -5.50020196e-03,  1.34151742e+00,  7.93102467e-01,  3.87893431e-02,
        8.53805503e-02, -3.79951121e+00, -1.50309931e-02,  3.22704891e-01,
       -4.57734276e-03,  5.36900921e-02,  1.89561453e-02, -1.24686473e-01,
        4.11771205e-03,  3.30938804e-02,  1.02860172e+00, -1.62540600e-02,
        6.53023690e-02,  8.26385688e-01, -1.70887788e-02,  9.49764489e-03,
       -8.64507413e-02, -